In [1]:
import numpy as np
import torch
import torch.nn as nn

from torch.autograd import Variable

import pyro
from pyro.distributions import Normal
from pyro.infer import SVI
from pyro.optim import Adam

In [43]:
N = 100
num_features = 1 
def build_linear_dataset(N, noise_std=0.01):
    X = np.linspace(-3,3, N)
    Y = X * 3 + 1 + np.random.normal(0, noise_std, size = N)
    X = torch.autograd.Variable(torch.Tensor(X.reshape((N, 1))))
    Y = torch.autograd.Variable(torch.Tensor(Y.reshape((N, 1))))
    return torch.cat((X, Y), 1)

In [36]:
class RegressionModel(nn.Module):
    def __init__(self, num_features):
        super(RegressionModel, self).__init__()
        self.linear = nn.Linear(num_features, 1)
        
    def forward(self, x):
        return self.linear(x)
    
    
regression = RegressionModel(num_features)


# Regression via point estimates

In [62]:
regression = RegressionModel(num_features)
loss_fn = torch.nn.MSELoss(False)
optimizer = torch.optim.Adam(regression.parameters(), lr=0.01)
iterations = 500

data = build_linear_dataset(N)
for i in range(iterations):
    output = regression(data[:, :-1])
    loss = loss_fn(output, data[:, 1])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (i + 1) % 50 == 0:
        print("Total Loss after %04d, %.4f" % (i + 1, loss.data[0]))
        
        
for name, param in regression.named_parameters():
    print("%s, %f" % (name, param.data.numpy()))

Total Loss after 0050, 2115.3093
Total Loss after 0100, 1410.7089
Total Loss after 0150, 914.0960
Total Loss after 0200, 573.1920
Total Loss after 0250, 345.9367
Total Loss after 0300, 199.9661
Total Loss after 0350, 110.3257
Total Loss after 0400, 57.9690
Total Loss after 0450, 28.9671
Total Loss after 0500, 13.7541
linear.weight, 2.789407
linear.bias, 1.000050


# Bayesian Regression

In [65]:
mu = torch.autograd.Variable(torch.zeros(1,1))
stddev = torch.autograd.Variable(torch.ones(1,1))

prior = Normal(mu, stddev)


lifted_module = pyro.random_module("regression_model", regression, prior)

sampled_model = lifted_module()

In [ ]:
def model(data):
    X = data[:, :-1]
    Y = data[:, -1]
    weight_mu = Variable(torch.zeros(1, num_features))
    weight_std = Variable(torch.ones(1, num_features) * 10)

    bias_mu = Variable(torch.zeros(1, num_features))
    bias_std = Variable(torch.ones(1, num_features) * 10)

    weight_prior = Normal(weight_mu, weight_std)
    bias_prior = Normal(bias_mu, bias_std)
    priors = { 'linear.weight': weight_prior, 'linear.bias': bias_prior }
    
    lifted_module = pyro.random_module("module", regression, priors)
    
    lifted_model = lifted_module()
    
    pred_mean = lifted_model(X).squeeze()
    
    pyro.sample("obs", Normal(pred_mean, Variable(0.1* torch.ones(X.size(0)))), obs=Y.squeeze())